In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

%matplotlib inline

# MNIST Dataset

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


In [ ]:
example_images, example_labels = mnist.train.next_batch(10)

plt.figure(figsize=(20,2))

for i in range(10):
    plt.subplot(1,10,i+1)
    plt.imshow(example_images[i,:].reshape((28,28)))
plt.show()

print(np.nonzero(example_labels)[1])


# Linear Regression

### Network and Loss function

In [ ]:
tf.reset_default_graph()

# input placeholder
x = tf.placeholder(tf.float32, [None, 784]) # 28**2 = 784

# weight vector
W = tf.Variable(tf.zeros([784, 10]))

# bias
b = tf.Variable(tf.zeros([10]))

# output layer
y = tf.nn.softmax(tf.matmul(x, W) + b)

# true labels
y_ = tf.placeholder(tf.float32, [None, 10])

# loss function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), axis=[1]))


### Optimization

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5)
train_step = optimizer.minimize(cross_entropy)

# tf.train.AdadeltaOptimizer
# tf.train.AdamOptimizer
# tf.train.RMSPropOptimizer
# ...

init_op = tf.global_variables_initializer()

### Evaluation

In [ ]:
# evaluation

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
print(cross_entropy)
print(x)
print(W)
print(accuracy)

### Run the experiment

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    
    summary_writer = tf.summary.FileWriter("./graph", graph=sess.graph)
    
    for _ in range(1000):
        # get a minibatch of size 100
        batch_xs, batch_ys = mnist.train.next_batch(100)
        
        # run the training step
        feed_dict = {x: batch_xs,
                     y_: batch_ys}
        sess.run(train_step, feed_dict=feed_dict)

    final_W = W.eval(session=sess)
    
    test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images,
                                                  y_: mnist.test.labels})
    
print(test_accuracy) 

In [ ]:
plt.figure(figsize=(20,2))

for i in range(10):
    plt.subplot(1,10,i+1)
    plt.imshow(final_W[:,i].reshape((28,28)))
plt.show()